In [25]:
import os

In [2]:
os.chdir("../")

In [26]:
%pwd

'c:\\Users\\reda-\\Desktop\\Projects\\MLOPS-Project'

In [27]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/RmBenjy/MLOPS-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "RmBenjy"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "ba8c84e91b9a47bfeb4b083b748f8e057a71c22a"

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path:Path
    model_path:Path
    all_params:dict
    metric_file_name:Path
    target_column:str
    mlflow_uri:str

In [30]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories, save_json

In [16]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath =  CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath =  SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column=schema.name,
            mlflow_uri = "https://dagshub.com/RmBenjy/MLOPS-Project.mlflow"
        )

        return model_evaluation_config

In [17]:
import os
import joblib
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np

In [31]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(X_test)

            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

            # Saving metrics as local
            scores = {"rmse":rmse, "mae":mae, "r2":r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model registry doesn't work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [32]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-10-08 20:04:27,487: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-08 20:04:27,488: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-08 20:04:27,493: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-08 20:04:27,496: INFO: common: created directory at: artifacts]
[2025-10-08 20:04:27,500: INFO: common: created directory at: artifacts/model_evaluation]
[2025-10-08 20:04:27,755: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Python\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ElasticnetModel'.
2025/10/08 20:05:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
